<a href="https://colab.research.google.com/github/joshfpedro/math-328/blob/main/least_squares_approximation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def least_squares_polynomial(x_data, y_data, degree):
    """
    Fit polynomial using least squares.

    Parameters
    ----------
    x_data : array
        x-coordinates of data
    y_data : array
        y-coordinates of data
    degree : int
        Degree of polynomial to fit

    Returns
    -------
    coefficients : array
        Polynomial coefficients [a_0, a_1, ..., a_m]
    """
    # Build Vandermonde matrix
    A = np.vander(x_data, degree + 1, increasing=True)

    # Solve normal equations: A^T A a = A^T y
    coeffs = np.linalg.solve(A.T @ A, A.T @ y_data)

    return coeffs

# Generate noisy data
np.random.seed(42)
x_data = np.linspace(0, 10, 20)
y_true_data = 2 + 0.5 * x_data - 0.05 * x_data**2
y_noisy = y_true_data + np.random.normal(0, 1.0, len(x_data))

# Test it
coeffs = least_squares_polynomial(x_data, y_noisy, 2)
print(f"Fitted polynomial: y = {coeffs[0]:.3f} + {coeffs[1]:.3f}x + {coeffs[2]:.3f}x²")
print(f"True polynomial:   y = 2.000 + 0.500x - 0.050x²")